In [1]:
#####load quanztized_resnet18
import sys
import time
import random
import os
sys.path.append("..")
from module.layer_fusion_module import *
from module.resnet_module import *
from module.extract_weight_module import *
model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
quantized_model_edited_filename = "q_resnet18_quantized_cifar10_edited.pt"
fusioned_model_filename = "q_fusioned_resnet18_cifar10.pt"
quantized_model_filename_jit = "q_resnet18_quantized_cifar10.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"

model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
quantized_model_filepath_jit = os.path.join(model_dir, quantized_model_filename_jit)
quantized_model_edited_filepath = os.path.join(model_dir, quantized_model_edited_filename)

num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model = create_model(num_classes=num_classes)
model.to(cpu_device)

quantized_model = QuantizedResNet18(model_fp32=model)

quantization_config = torch.quantization.get_default_qconfig("fbgemm")

quantized_model.qconfig = quantization_config
fuse_model(quantized_model)
torch.quantization.prepare_qat(quantized_model, inplace=True)
quantized_model.to(cpu_device)    
quantized_model = torch.quantization.convert(quantized_model, inplace=True)
#save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename_jit)
#save_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)
loaded_model=load_model(model=quantized_model, model_filepath=quantized_model_edited_filepath, device='cpu')
loaded_model.eval()
#save each layer weights


{'model.model_fp32.conv1': [['0', '1', '2']], 'model.model_fp32.conv2_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv2_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv3_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv3_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv3_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv4_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv4_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv4_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv5_x[0].residual_function': [['0', '1', '2'], ['3', '4', '5']], 'model.model_fp32.conv5_x[0].shortcut': [['0', '1', '2']], 'model.model_fp32.conv5_x[1].residual_function': [['0', '1', '2'], ['3', '4', '5']]}


C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:174: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
C:\Users\win0\Anaconda3\envs\pytorch\lib\site-packages\torch\ao\quantization\observer.py:1109: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  Returning default scale and zero point "


QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0374]), zero_point=tensor([57]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): ResNet(
    (conv1): Sequential(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.05000000074505806, zero_point=0, padding=(3, 3))
      (1): Identity()
      (2): Identity()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_x): Sequential(
      (0): q_BasicBlock(
        (residual_function): Sequential(
          (0): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.0312686413526535, zero_point=0, padding=(1, 1))
          (1): Identity()
          (2): Identity()
          (3): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.05000000074505806, zero_point=0, padding=(1, 1))
          (4): Identity()
          (5): Identity()
        )
        (shortcut): Sequential()
        (q_func): QFunctional(
          s

In [3]:
import sys
import time
import random
import os
sys.path.append("..")
from module.make_cfg_module import *
input_width=32
input_height=32
input_channel=3
mean_list=list()
mean_list.append(0.485)
mean_list.append(0.456)
mean_list.append(0.406)
std_list=list()
std_list.append(0.229)
std_list.append(0.224)
std_list.append(0.225)
make_cfg_from_pytorch(quantized_model,"edited_resnet18.cfg",input_width,input_height,input_channel,mean_list,std_list)

[net]

batch=1

subdivisions=1

max_batches=1

momentum=0.9

decay=0.00005

policy=poly

power=4

learning_rate=0.01

angle==1

hue==1

saturation=1

exposure=1

aspect=1

height=32

width=32

channels=3

quantization_type=1

start_check_point=0

end_check_point=0

input_scale=0.03739064

input_zeropoint=57

normalize_mean_0=0.485

normalize_mean_1=0.456

normalize_mean_2=0.406

normalize_var_0=0.229

normalize_var_1=0.224

normalize_var_2=0.225


[convolutional]

quantization_type=1

quantization_layer_scale=0.05000000074505806

quantization_layer_zeropoint=0

filters=64

size=7

pad=3

stride=2

activation=relu


[maxpool]

quantization_type=1

quantization_layer_scale=0.05000000074505806

quantization_layer_zeropoint=0

size=3

pad=1

stride=2


[convolutional]

quantization_type=1

quantization_layer_scale=0.0312686413526535

quantization_layer_zeropoint=0

filters=64

size=3

pad=1

stride=1

activation=relu


[convolutional]

quantization_type=1

quantization_layer_scale=0.050000